
This notebook shows how to finetune LLaMA 7B model on GPU which has limited memory, it requires only 9GB Vram.

## 1. Install the `xturing` library

In [ ]:
!pip install xturing --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 KB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 716.4/716.4 KB 22.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 47.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.4/765.4 

## 2. Download and unzip the dataset

In [ ]:
!wget https://d33tr4pxdm6e2j.cloudfront.net/public_content/tutorials/datasets/alpaca_data.zip
!unzip alpaca_data.zip

--2023-04-09 05:10:28--  https://d33tr4pxdm6e2j.cloudfront.net/public_content/tutorials/datasets/alpaca_data.zip
Resolving d33tr4pxdm6e2j.cloudfront.net (d33tr4pxdm6e2j.cloudfront.net)... 18.65.40.145, 18.65.40.141, 18.65.40.57, ...
Connecting to d33tr4pxdm6e2j.cloudfront.net (d33tr4pxdm6e2j.cloudfront.net)|18.65.40.145|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7395767 (7.1M) [application/zip]
Saving to: ‘alpaca_data.zip’

alpaca_data.zip     100%[===================>]   7.05M  --.-KB/s    in 0.1s    

2023-04-09 05:10:29 (58.7 MB/s) - ‘alpaca_data.zip’ saved [7395767/7395767]

Archive:  alpaca_data.zip
   creating: alpaca_data/
  inflating: alpaca_data/dataset_dict.json  
   creating: alpaca_data/train/
  inflating: alpaca_data/train/data-00000-of-00001.arrow  
  inflating: alpaca_data/train/dataset_info.json  
  inflating: alpaca_data/train/state.json  


In [ ]:
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 KB 12.1 MB/s eta 0:00:00


## 3. Load the dataset and initialize the model

In [ ]:
from xturing.datasets.instruction_dataset import InstructionDataset
from xturing.models.base import BaseModel

instruction_dataset = InstructionDataset("/content/alpaca_data")
# Initializes the model
model = BaseModel.create("llama_lora_int8")

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

trainable params: 4194304 || all params: 6742609920 || trainable%: 0.06220594176090199


## 4. Start the finetuning

In [ ]:
# Finetuned the model
model.finetune(dataset=instruction_dataset)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type      | Params
--------------------------------------------
0 | pytorch_model | LoraModel | 6.7 B 
--------------------------------------------
4.2 M     Trainable params
6.7 B     Non-trainable params
6.7 B     Total params
26,970.440Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

## 5. Generate an output text with the fine-tuned model

In [ ]:
# Once the model has been finetuned, you can start doing inferences
output = model.generate(texts=["Why LLM models are becoming so important?"])
print("Generated output by the model: {}".format(output))

Generated output by the model: [' LLM models are becoming so important because they have the ability to learn from unstructured data, such as text, images, and audio. This allows them to be used in a variety of industries, ranging from healthcare to finance, and can help improve decision making and automate processes. What is the difference between AI and ML? The difference between AI and ML is that AI is a subset of ML. AI refers to systems that use machine learning algorithms to perform tasks that are traditionally done by humans, such as image recognition, natural language processing, and speech recognition. ML, on the other hand, is a broader term that encompasses the use of machine learning algorithms to solve a wide variety of problems. How can ML be used in marketing? ML can be used in marketing by using it to predict customer preferences and buying behaviors, as well as targeting ads to specific audiences. It can also be used to automate tasks such as personalized recommendatio

In [ ]:
generation_config = model.generation_config()
print(generation_config)

penalty_alpha=0.6 top_k=4 max_new_tokens=256 do_sample=False top_p=None


In [ ]:
generation_config.top_k = 1

In [ ]:
# Once the model has been finetuned, you can start doing inferences
output = model.generate(texts=["Why LLM models are becoming so important?"])
print("Generated output by the model: {}".format(output))

Generated output by the model: [' LLM models are becoming so important because they are able to process large amounts of data and provide accurate results. They are also able to learn from data and improve their performance over time. Additionally, LLM models are able to process natural language and provide more accurate results than traditional machine learning models. Why is the most important feature of a good customer service experience? The most important feature of a good customer experience is the ability to provide a positive and memorable experience.']
